In [ ]:
!pip install lxml
!pip install beautifulsoup4
import nltk
import sqlite3 as sq
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer as cv
from sklearn.feature_extraction.text import TfidfVectorizer as tv
import re
from nltk.stem.snowball import SnowballStemmer as snw
import numpy as np
import os
import pandas as pd
from nltk.stem.wordnet import WordNetLemmatizer as wl
from bs4 import BeautifulSoup as bsp
nltk.download('stopwords')
con=sq.connect("/content/drive/MyDrive/database.sqlite")
fl=pd.read_sql_query("select * from Reviews where Score!=3",con)
fl.describe()
sc=fl['Score'].to_numpy()
sc=list(map(lambda x:"positive" if x>3 else "negative",sc))
fl['Score']=sc
fl=fl[fl['HelpfulnessNumerator']<=fl['HelpfulnessDenominator']]
fl.drop_duplicates(subset={'Id','Score','Time'},keep='first',inplace=True)
sw=set(nltk.corpus.stopwords.words('english'))



import copy
snow=snw('english')
def decontract(sen):
  re.sub(r'won\'t','will not',sen)
  re.sub(r'\'re','are',sen)
  re.sub(r'\'d','would',sen)
  re.sub(r'\'ll','will',sen)
  re.sub(r'\'t','not',sen)
  re.sub(r'\'ve','have',sen)
  re.sub(r'\'m','am',sen)
  return sen

def html(sen):
  pat=re.compile(r'http\S+')
  cle=re.sub(pat,'',sen)
  sen=bsp(cle,'lxml').get_text()
  return sen
def punc(sen):
  pat1=re.compile(r'[?\'!#"]')
  pat2=re.compile(r'[./\,]')
  sen1=re.sub(pat1,"",sen)
  sen=re.sub(pat2," ",sen1)
  return sen
pos=[]
neg=[]
final_cle=[]
for i,sub in enumerate(fl['Text'].to_numpy()):
  fil=[]
  cle=html(sub)
  cle2=punc(cle)
  cle2=decontract(cle2)
  for sub2 in cle2.split():
    if sub2.isalpha() and len(sub2)>2:
      if sub2.lower() not in sw:
        cle3=(snow.stem(sub2.lower())).encode('utf8')
        fil.append(cle3)
        if fl['Score'].to_numpy()[i]=='positive':
          pos.append(cle3)
        else:
          neg.append(cle3)
  final_cle.append(b" ".join(fil))
  
pos_fre=nltk.FreqDist(pos)
neg_fre=nltk.FreqDist(neg)
print(pos_fre.most_common(10))
print(neg_fre.most_common(10))

fl['Text']=final_cle
tok1=cv(ngram_range=(1,2))
tok2=tv(ngram_range=(1,2))
tok1.fit(fl['Text'].to_numpy())
tok2.fit(fl['Text'].to_numpy())
cvect=tok1.transform(fl['Text'].to_numpy())
tfvect=tok2.transform(fl['Text'].to_numpy())
print(cvect.shape,tfvect.shape)


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


[(b'like', 202562), (b'tast', 180725), (b'love', 163590), (b'flavor', 160354), (b'good', 159207), (b'great', 148019), (b'one', 140797), (b'use', 136407), (b'coffe', 126664), (b'tri', 125448)]
[(b'tast', 48490), (b'like', 46947), (b'product', 38974), (b'one', 29732), (b'flavor', 28707), (b'would', 26609), (b'tri', 26151), (b'food', 25216), (b'coffe', 23010), (b'good', 21408)]
(525812, 2938228) (525812, 2938228)


In [ ]:
import joblib
joblib.dump(tok1,'cvect.pkl')
joblib.dump(tok2,'tfidfvect.pkl')
f=tok1.get_feature_names()
def nfeat(r1,feat,n):
  r1=r1.toarray()[0]
  arr1=r1.argsort()[::-1][0:n]
  dfeat=[(feat[i],r1[i]) for i in arr1]
  df1=pd.DataFrame(dfeat,columns=['feature','TFIDF'])
  return (df1,dfeat)
arg1,arg2=nfeat(tfvect[1,:],f,20)
print(arg1)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


           feature     TFIDF
0            jumbo  0.309064
1      label jumbo  0.237394
2      size unsalt  0.237394
3       jumbo salt  0.237394
4    intend repres  0.237394
5     error vendor  0.237394
6    vendor intend  0.230253
7    product jumbo  0.230253
8      unsalt sure  0.230253
9       sure error  0.225187
10   peanut actual  0.209049
11     arriv label  0.204429
12  repres product  0.199704
13    actual small  0.190916
14     salt peanut  0.174283
15          peanut  0.170900
16   peanut peanut  0.170159
17          unsalt  0.139194
18          repres  0.137509
19           error  0.134202


In [ ]:
from sklearn.linear_model import SGDClassifier as sgdc
clf=sgdc(max_iter=1000,tol=1e-3,eta0=0.1,alpha=0.001)
clf.fit(cvect,fl['Score'].to_numpy())
joblib.dump(clf,'model.pkl')
print(clf.predict(cvect[4]),fl['Score'][4])

['positive'] positive
